# ___
# [ Geoinformatika ]

## Strojové učení


**Katedra aplikované geoinformatiky a kartografie**, Přírodovědecká fakulta, Univerzita Karlova
Albertov 6, 128 43 Praha 2

*Lukáš Brodský lukas.brodsky@natur.cuni.cz*

### Rozhodovací strom *(Decision tree)* v Pythonu 


Tento Jupyter notbooku demonstruje strojové učení s využitím algoritmu rozhodovací strom z knihovny Scikit learn (*https://scikit-learn.org*). 


### Řešený problem
Klasifikace druhů kosatců na základě jejich vlastností (délka a šířka okvětních a kališních lístků).  

### Měření míry úspěchu

Strojové učení používá podobné metriky jako DPZ. Pojmenování se trochu liší. 

Specificita (*precision*) obecně říká, kolik vzorků označených za pozitivních je opravdu pozitivních.

$$ precision = \frac{TP}{TP+FP} $$

TP označuje počet správně označených pozitivních vzorků (*true positives*)
FP označuje počet falešně pozitivních vzorků (*false negatives*)

Senzitivita (*recall*) testu obecně vyjadřuje kolik pozitivních vzorků bylo podchyceno klasifikátorem (klasifikováno jako pozitivní).


$$ recall = \frac{TP}{TP+FN} $$

FN je počet falešně negativních vzorků (*false negatives*)


Zda je pro nás důležitější *precision* nebo *recall*, záleží na konkrétní úloze. Někdy vadí více falešně pozitivní případy (např. příliš mnoho relevantních mailů označených za spam), jindy bude více vadit nezachycený pozitivní případ (např. neodhalený výskyt nemoci).

Další často používanou metrikou je tzv. **F1 skóre**. Kombinuje *precision* a *recall*, a to tak, že obě tyto metriky mají stejnou váhu (přizpívají stejnou měrou k výsledku). Čím větší hodnota, tím lepší výsledek. Maximální hodnota je jedna, minimální 0.

$$ F1 = 2 \frac{precision \cdot recall}{precision + recall}$$



S použitím F1-skóre balancujeme *precision* a *recall* (harmonický průměr). Tuto metriku vybereme pro hodnocení úspěšnosti našeho modelu pro tento problem!  


In [ ]:
# Knihovny

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Jupyter notebook vizualizace
%matplotlib inline
import seaborn as sns

# kodovani 
from sklearn.preprocessing import LabelEncoder
# rozdeleni na trenovaci a testovaci dataset 
from sklearn.model_selection import train_test_split
# trida klasifikatoru rozhodovaci strom 
from sklearn.tree import DecisionTreeClassifier
# validacni indikatory a procedury 
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, confusion_matrix
# vizualizace stromu 
from sklearn.tree import plot_tree
from matplotlib.pyplot import figure
import matplotlib.image as mpimg

### Nacteni vstupnich dat

In [ ]:
# vstupni data
df = sns.load_dataset('iris') 
df.head()

### Explorace vstupnich dat 

In [ ]:
# zakladni info 
df.info()

In [ ]:
# vizualizace kvetu

img = 'iris-dataset.png'
img = mpimg.imread('iris-dataset.png')
plt.figure(figsize=(15, 10), dpi=80)
plt.imshow(img)
# okvětní lístek a kališní lístek 

In [ ]:
# dimenzionalita matice 
df.shape

In [ ]:
# chybejici atributy? 
df.isnull().any()

In [ ]:
# vizualizace atributu - pruzkumova analyza 
sns.pairplot(data=df, hue = 'species')

In [ ]:
# existuji korelace (linerani zavislosti) v matici priznaku?
# sns.heatmap(df.corr(), cmap="Greens")

## Priprava dat

In [ ]:
df.columns

In [ ]:
# klasifikacni tridy 
df['species']

In [ ]:
reference = df['species']
df1 = df.copy()
df1 = df1.drop('species', axis = 1)

In [ ]:
# Defining the attributes 
X = df1

In [ ]:
X

In [ ]:
# reference

In [ ]:
# kodovani trid 
le = LabelEncoder()
cis_kody = le.fit_transform(reference) 
cis_kody

In [ ]:
# kodovani 
spec_kod  = pd.concat([df['species'], pd.DataFrame(cis_kody)], axis=1)

for col in spec_kod:
    print(spec_kod[col].unique())

In [ ]:
y = cis_kody

### Rozdeleni na trenovaci a testovaci dataset 

In [ ]:
# nahodne rozdeleni 
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size = 0.9, random_state = 42)

print("Trenovaci mnozina ", X_train.shape)
print("Testovaci mnozina ", X_test.shape)

### Modelovani a testovani stromu

In [ ]:
# vytvor instanci tridy pro rozhodovaci strom 
# a nastav rucne hyperparametry: max. depth, min. samples, splitter
strom = DecisionTreeClassifier(max_depth=3, min_samples_split=2, splitter='random', random_state = 42) 

In [ ]:
### prvni "fitovani" modelu
strom.fit(X_train, y_train)

In [ ]:
# prvni validace na testovacim datasetu
round(f1_score(y_test, strom.predict(X_test), average='macro'), 3)

In [ ]:
# krizova validace stromu 
cv_strom = cross_validate(strom, X_train,y_train, cv=5, scoring='f1_macro', return_estimator=True)
print('Krizova validace!')

In [ ]:
# cv_strom['test_score']

In [ ]:
print('Prumer vazeneho F1-skore: {:.3f} a std {:.3f}'.format(
        cv_strom['test_score'].mean(),
        cv_strom['test_score'].std())
     )

In [ ]:
y_pred = strom.predict(X_test)

In [ ]:
# konfuzni matice 
cm = confusion_matrix(y_test, y_pred) 
plt.figure(figsize=(7,7))
sns.heatmap(data=cm,linewidths=.5, annot=True, square=True, cmap='Blues')
plt.ylabel('Reference')
plt.xlabel('Predikovana trida')
all_sample_title = 'F1-score: {0}'.format(round(f1_score(y_test, y_pred, average='macro'), 3))
plt.title(all_sample_title, size = 15)

In [ ]:
# Visualizace stromu 
figure(figsize=(9, 6), dpi=80)
rozhodovaci_strom = plot_tree(decision_tree=strom, feature_names = df1.columns,
class_names =["setosa", "vercicolor", "verginica"] , filled=True ,  precision=2, rounded=True)

In [ ]:
# stratifikace
# from sklearn.model_selection import StratifiedKFold
# skf = StratifiedKFold(n_splits=5, shuffle=True)
# skf.get_n_splits(X, y)
# for train_index, test_index in skf.split(X, y):
    # print("TRAIN:", train_index)
    # print("TEST:", test_index)


In [ ]:
# Jiny algoritmus klasifikace? 
# David Wolpert (1996) - "No Free Lunch Therem" - if you make absolutely no assumption about the data, 
# there is no reason to prefer one model over any other!

In [ ]:
# df_vv = df[df['species'] != 'setosa']

In [ ]:
# vizualizace atributu - pruzkumova analyza 
# sns.pairplot(data=df_vv, hue = 'species')

In [ ]:
# sns.set(rc = {'figure.figsize':(10,7)})
# sns.scatterplot(data=df_vv, x="petal_length", y="petal_width", hue="species")

In [ ]:
# chyby = (y_test - y_pred) != 0

In [ ]:
# viz_chyby = pd.DataFrame.copy(X_test)
# viz_chyby['species'] = y_test
# viz_chyby['chyby'] = chyby

In [ ]:
# sns.scatterplot(data=viz_chyby, x="petal_length", y="petal_width", hue="species", style="chyby") 